In [2]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import pandas
from sklearn.metrics import mean_squared_error

In [3]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [6]:
answers = {}

In [7]:
def UsersGames(data):
    usergames = defaultdict(set)
    gameusers = defaultdict(set)
    for u, g, d in data:
        usergames[u].add(g)
        gameusers[g].add(u)
    return usergames, gameusers

In [10]:
allHours = []
for l in readJSON("/Users/zhiqiaogong/Projects/JupyterNotebook/cse258/hw3/train.json.gz"):
    allHours.append(l)

In [11]:
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

In [748]:
# Evaluate baseline strategy
allGames = set([g for u, g, d in hoursTrain + hoursValid])

usergames,gameusers = UsersGames(hoursTrain + hoursValid)

augmentedValidationSet = []

valid1 = []
valid2 = []

for u,i,d in hoursValid:
    valid1.append((u, i, 1))
    valid2.append((u, random.choice(list(allGames - usergames[u])), 0))
augmentedValidationSet = valid1 + valid2

In [162]:
##################################################
# Hours played prediction                        #
##################################################

In [12]:
trainHours = [r[2]['hours_transformed'] for r in hoursTrain]
globalAverage = sum(trainHours) * 1.0 / len(trainHours)

In [13]:
### Question 6

In [14]:
hoursPerUser = {}
hoursPerItem = {}
for u, g, d in hoursTrain:
    t = d.get('hours_transformed')
    if u not in hoursPerUser:
        hoursPerUser[u] = set()
    hoursPerUser[u].add(t)
    if g not in hoursPerItem:
        hoursPerItem[g] = set()
    hoursPerItem[g].add(t)

In [15]:
betaU = {}
betaI = {}
for u in hoursPerUser:
    betaU[u] = 0

for g in hoursPerItem:
    betaI[g] = 0

In [16]:
alpha = globalAverage # Could initialize anywhere, this is a guess
alpha

3.716088074007024

In [17]:
lambda_ = 1
iter = 100
tolerance = 1e-5 
pmse = 0
converged = False

In [18]:
usergameHours = {}
for u, g, d in hoursTrain:
    usergameHours[(u,g)] = d['hours_transformed']

In [21]:
usergames_train,gameusers_train = UsersGames(hoursTrain)

In [19]:
def iterate(lambu,lambi):
    alpha = globalAverage
    betau_iterate = {}
    betai_iterate = {}
    for u in hoursPerUser:
        betau_iterate[u] = 0
    
    for g in hoursPerItem:
        betai_iterate[g] = 0
    iter = 100
    tolerance = 1e-8
    pmse = float("inf")
    converged = False
    tolerance_count=3
    
    for iteration in range(iter):
        alpha = sum([ usergameHours[d] - betau_iterate[d[0]] - betai_iterate[d[1]] for d in usergameHours])/len(usergameHours)
        
        for u in usergames_train:
            betau_iterate[u] = sum([ usergameHours[(u,g)] - alpha - betai_iterate[g] for g in usergames_train[u]])/(lambu + len(usergames_train[u]))
    
        for g in gameusers_train:
            betai_iterate[g] = sum([ usergameHours[(u,g)] - alpha - betau_iterate[u] for u in gameusers_train[g]])/(lambi + len(gameusers_train[g]))
    
        predictions = [alpha + betau_iterate[u] + betai_iterate[g] for u,g,d in hoursValid]
        y = [d['hours_transformed'] for u,g,d in hoursValid]
    
        mse = mean_squared_error(predictions, y)
        
        if mse > pmse:
            break
        else:
            pmse = mse

    return alpha, betau_iterate, betai_iterate, mse

In [27]:
_,_,_,t = iterate(4.8,3.4)
t

2.9885787519794094

In [48]:
# Better lambda...
bestmse=float("inf")
bestlambu = 7.48
bestlambi = 0
for li in range(210,230,1):
    _,_,_,t = iterate(bestlambu,li/100)
    print(t,li)
    if t < bestmse:
        bestmse = t
        bestlambi = li/100
    
bestmse,bestlambi

2.985433074009893 210
2.9854386660552032 211
2.9854443919860323 212
2.985450251342035 213
2.98545624366471 214
2.985462368497393 215
2.9854686253852485 216
2.9854750138752597 217
2.985481533516219 218
2.9854881838587213 219
2.9851319564935865 220
2.9851321236195996 221
2.985132455228408 222
2.9851327955425937 223
2.9851332308064475 224
2.9851337607687034 225
2.985134450106289 226
2.985135151917563 227
2.985135947683033 228
2.9851368371583655 229


(2.9851319564935865, 2.2)

In [41]:
# Better lambda...
bestmse=float("inf")
bestlambu = 0
bestlambi = 2.2
for lu in range(740,750,1):
    _,_,_,t = iterate(lu/100,bestlambi)
    print(t,lu)
    if t < bestmse:
        bestmse = t
        bestlambu = lu/100
    
bestmse,bestlambu

2.985150640428202 740
2.9851481287280572 741
2.9851456676179566 742
2.9851432569750873 743
2.985140896676962 744
2.9851385866014217 745
2.985136326626632 746
2.9851341166310825 747
2.9851319564935865 748
2.9854927193395175 749


(2.9851319564935865, 7.48)

In [38]:
bestmse,bestlambu

(2.986299838993884, 5.8)

In [31]:
bestmse,bestlambi

(2.9877543109555837, 2.2)

In [25]:
# Better lambda...
bestmse=float("inf")
bestlambu = 0
bestlambi = 0
for lu in range(400,500,20):
    for li in range(340,440,20):
        _,_,_,t = iterate(lu/100,li/100)
        print(t,li,lu)
        if t < bestmse:
            bestmse = t
            bestlambi = li/100
            bestlambu = lu/100
    
bestmse

2.990605004214618 340 400
2.9908202125682863 360 400
2.9910645213183185 380 400
2.991336689786135 400 400
2.991635529915669 420 400
2.9900317164306576 340 420
2.9902466176410147 360 420
2.990490577401678 380 420
2.9907623574701256 400 420
2.9910607720837663 420 420
2.9895045147489765 340 440
2.9897191179528555 360 440
2.989962739112359 380 440
2.990234142357712 400 440
2.990532144159605 420 440
2.989020956874923 340 460
2.98923527081857 360 460
2.9894785633228116 380 460
2.9897496008297577 400 460
2.9900472019855244 420 460
2.9885787519794094 340 480
2.9887927850420177 360 480
2.989035758420841 380 480
2.9893064408113394 400 480
2.989603652979412 420 480


2.9885787519794094

In [26]:
bestmse

2.9885787519794094

In [49]:
alpha, betaU, betaI, mse5 = iterate(7.48,2.2)
mse5

2.9851319564935865

In [841]:
answers['Q8'] = (l, bestmse)

In [842]:
assertFloatList(answers['Q8'], 2)

In [46]:
predictions = open("/Users/zhiqiaogong/Projects/JupyterNotebook/cse258/assignment1/test/predictions_Hours.csv", 'w')
for l in open("/Users/zhiqiaogong/Projects/JupyterNotebook/cse258/hw3/pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    bu = betaU[u]
    bi = betaI[g]
    
    _ = predictions.write(u + ',' + g + ',' + str(alpha + bu + bi) + '\n')

predictions.close()

In [935]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()